#Запуск алгоритма

In [13]:
#@title Установка пакетов
!pip  install langchain==0.0.335 openai==1.2.3 tiktoken==0.5.1 pydantic==1.10.8 faiss-cpu==1.7.4 nltk oauth2client >/dev/null
!pip install gspread

In [14]:
#@title Импорт библиотек

import os
import getpass
import requests

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import openai
from openai import OpenAI
import tiktoken
import re
import requests
from langchain.docstore.document import Document

import pandas as pd
from google.oauth2.service_account import Credentials

from oauth2client.service_account import ServiceAccountCredentials
import gspread
import json
import numpy as np


# для memory
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.llms import OpenAI as lc_OpenAI


In [15]:
#@title Опредение функций

def load_text_from_github(kdb_link):       # Функция предназначена для загрузки текста из гитхаба, по ссылке (kdb_link)
  response = requests.get(kdb_link)
  txt = response.text
  return txt

def load_googledoc_by_url(doc_url) -> str: # Функция предназначена для загрузки текста из гуглдока, по ссылке (doc_url)
  # Extract the document ID from the URL
  match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', doc_url)
  if match_ is None:
    raise ValueError('Invalid Google Docs URL')
  doc_id = match_.group(1)

  # Download the document as plain text
  response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
  response.raise_for_status()
  return response.text

def load_document_text(file_path) -> str:   # Функция load_document_text предназначена для загрузки текста из файла, расположенного по указанному пути (file_path)
#    with open(file_path, 'r', encoding='windows-1251') as file:
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    text_encoded = text.encode('utf-8')
    text = text_encoded.decode('utf-8')
    return text

def load_text(any_link):
  if len(any_link)==0:
    text=''
  else:
    if "github.com" in any_link:
      if "blob" in any_link: any_link=any_link.replace("blob", "raw")
      text = load_text_from_github(any_link)
    elif "docs.google.com" in any_link:
      text = load_googledoc_by_url(any_link)
    else:
      text = load_document_text(any_link)
    return text


def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

def split_text(text, max_count, chunk_overlap, verbose=0, double_split=1):
    # Функция для подсчета количества токенов в фрагменте
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
                          ]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    fragments = markdown_splitter.split_text(text)

    if double_split:
      # Создание объекта разделителя текста
      splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=chunk_overlap, length_function=num_tokens)

    # Список для хранения фрагментов текста
    source_chunks = []

    # Обработка каждого фрагмента текста
    for fragment in fragments:

        if verbose:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose
            count = num_tokens(fragment.page_content)
            print(f"Tokens in text fragment = {count}\n{'-' * 5}\n{fragment.page_content}\n{'=' * 20}")
        if double_split:
          # Разбиение фрагмента текста на части заданной длины с помощью разделителя
          # и добавление каждой части в список source_chunks  и передача в чанк метадата из маркдауновскго сплиттера
          source_chunks.extend(Document(page_content=chunk, metadata=fragment.metadata) for chunk in splitter.split_text(fragment.page_content))

    # Возвращение списка фрагментов текста
    if double_split:
      return source_chunks
    else:
      return fragments

def create_search_index(data, chunk_size, chunk_overlap, verbo, double_split):
    source_chunks = []
    source_chunks = split_text(text=data, max_count=chunk_size, chunk_overlap=chunk_overlap, verbose=verbo, double_split=double_split)
    return FAISS.from_documents(source_chunks, OpenAIEmbeddings())

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start> assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def answer_user_question(system_doc_text, knowledge_base_url, topic, instructions, temperature, verbose, k, chunk_size, chunk_overlap, model, double_split, history=''):
    knowledge_base_text = load_text(knowledge_base_url)
    knowledge_base_index = create_search_index(knowledge_base_text, chunk_size, chunk_overlap, verbose, double_split)
    return answer_index(system_doc_text, topic, instructions, knowledge_base_index, temperature, verbose, k, model, history)


def answer_index(system, topic, instructions, search_index, temp, verbose, k, model,hist=''):
    docs = search_index.similarity_search_with_score(topic, k=k)
#    if len(hist)>3: instructions += 'Запрещено приветствие клиента, только продолжай беседу.'
    message_content = '\n '.join([f'Отрывок текста №{i+1}\n{doc[0].page_content}' for i, doc in enumerate(docs)])
    messages = [{"role": "system", "content": system}, {"role": "user", "content": f"{instructions}\n\nТексты для анализа:\n{message_content}\n\nИстория чата:\n{hist}\n\nВопрос клиента: {topic}"}]

    completion = openai.chat.completions.create(model=model, messages=messages, temperature=temp)
    return message_content, completion.choices[0].message.content


#Получение ключа API

In [16]:
#@title Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Тест моделей

In [24]:
#@title 1. Тест по вопросам к БЗ - вопрос задаем вручную с поддержкой памяти
model = "gpt-3.5-turbo-1106" #@param ["gpt-3.5-turbo", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-1106"]
knowledge_base_link = "https://github.com/terrainternship/GPT_LaserLove/blob/main/knowledge.md?raw=true" #@param {type:"string"}
temperature = 0.4 #@param {type: "slider", min: 0, max: 1, step:0.1}
num_fragment = 5 #@param {type:"integer"}
verbose = "0" #@param [0,1]
system_prompt_link= "https://github.com/terrainternship/GPT_LaserLove/raw/main/%D0%9F%D0%A0%D0%9E%D0%9C%D0%A2%20LaserLove%20Svetl.txt" #@param {type:"string"}
instructions_link = "https://docs.google.com/document/d/18BARvMyB0ZZ0LhAwdxyb_PSeJ0k9Rhg1hZuKr2dLeoc" #@param {type:"string"}
#Вопрос_клиента = "\u041A\u0430\u043A\u043E\u0439 \u0441\u043E\u0441\u0442\u0430\u0432 \u0443 \u043C\u043E\u043B\u043E\u0447\u043A\u0430 \u0441 \u044D\u0444\u0444\u0435\u043A\u0442\u043E\u043C \u043B\u0438\u0444\u0442\u0438\u043D\u0433\u0430?" #@param {type:"string"}

system_prompt = load_text(system_prompt_link)
instructions = load_text(instructions_link)

history = ChatMessageHistory()
Вопрос_клиента = ''
memory_prompt_template = '''
if the Human introduced themselves (mentioned their name), make sure to include his name in the summary.
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think it is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.

New lines of conversation:
Human:What do you think about climate change?
AI: The discussion on climate change often involves diverse perspectives, strategies, and approaches to address the issue.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human shifts the topic and inquires about climate change. AI suggests that this question includes different points of view, strategies and approaches to solving the problem

END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
'''
while True:

  Вопрос_клиента = input('Вопрос клиента: ')
  if Вопрос_клиента.lower() == 'stop' or Вопрос_клиента.lower() == 'стоп':
      break

  # Извлечение текста сообщений из history
  user_questions_from_history = [message.content for message in history.messages]
  # Объединение текста в одну строку
  user_questions_string_from_history = ' '.join(user_questions_from_history)
# если в параметр history подать user_questions_string_from_history, то это история без саммари
# если подать buffer, то это только саммаризация предыдущ диалога на английском (имя сохраняет на английском)

  chunks, output1 = answer_user_question(system_prompt, knowledge_base_link, Вопрос_клиента,
                               instructions, temperature, int(verbose), num_fragment,
                               0, 0, model, double_split=0,history=memory.buffer)
  history.add_user_message(Вопрос_клиента)
  history.add_ai_message(output1)
  memory = ConversationSummaryMemory.from_messages(
    llm=lc_OpenAI(temperature=0),
    chat_memory=history,
    return_messages=True,
    prompt_template=memory_prompt_template
    )
  print("\nОтвет:\n", output1)




Вопрос клиента: привет, это Вероника, расскажи про лпг

Ответ:
 Привет, Вероника! Рада приветствовать тебя! ЛПГ (Липомассаж по Годе) - это инновационная процедура, которая помогает не только корректировать фигуру, но и улучшать состояние кожи. 

Во-первых, ЛПГ помогает повысить тургор атоничной кожи, устранить морщины и улучшить состояние кожи. Это не просто процедура для фигуры, а полноценный уход за кожей.

Во-вторых, цены на сеансы ЛПГ массажа могут отличаться в зависимости от уровня аппарата и клиники. Но, по сравнению с ручным массажем, цена такая же, иногда даже ниже. В клиниках сети Laser Love можно приобрести абонементы на массаж ЛПГ со скидкой. Также действуют специальные условия на первую процедуру.

И, наконец, эффективность ЛПГ массажа подтверждается многолетним опытом. Ключевой эффект – улучшение кровоснабжения тканей, нормализация лимфотока, избавление от отеков. Для достижения выраженного результата важно соблюдать рекомендации специалиста и пройти курс из 8-15 процедур.

In [28]:
print(user_questions_string_from_history)
print(f'\nmemory.buffer: {memory.buffer}')

привет, это Вероника, расскажи про лпг Привет, Вероника! Рада приветствовать тебя! ЛПГ (Липомассаж по Годе) - это инновационная процедура, которая помогает не только корректировать фигуру, но и улучшать состояние кожи. 

Во-первых, ЛПГ помогает повысить тургор атоничной кожи, устранить морщины и улучшить состояние кожи. Это не просто процедура для фигуры, а полноценный уход за кожей.

Во-вторых, цены на сеансы ЛПГ массажа могут отличаться в зависимости от уровня аппарата и клиники. Но, по сравнению с ручным массажем, цена такая же, иногда даже ниже. В клиниках сети Laser Love можно приобрести абонементы на массаж ЛПГ со скидкой. Также действуют специальные условия на первую процедуру.

И, наконец, эффективность ЛПГ массажа подтверждается многолетним опытом. Ключевой эффект – улучшение кровоснабжения тканей, нормализация лимфотока, избавление от отеков. Для достижения выраженного результата важно соблюдать рекомендации специалиста и пройти курс из 8-15 процедур. После курса кожа станови